Extract the data of the current season from http://www.football-data.co.uk/, Transform them and Load them in a Database.

In [1]:
# Import library
import json
import sqlite3
import requests
import io
import pandas as pd # data processing, CSV file I/O
from IPython.display import display # Manage multiple output per cell

In [2]:
# DB Sqlite connection
db = "/Users/thibaultclement/Project/ligue1-predict/src/notebook/data/db/soccer_predict.sqlite"
conn = sqlite3.connect(db)
cur = conn.cursor()

In [3]:
# Clean Database
cur.execute('DROP TABLE IF EXISTS cur_season_matchs_raw')
cur.execute('DROP TABLE IF EXISTS cur_season_pre_matchs')
conn.commit()

In [4]:
# Configuration
leagues = ['D1', 'E0', 'E1', 'E2', 'F1', 'I1', 'SP1', 'SC0']
seasons = ['1718']
website = "http://www.football-data.co.uk/mmz4281"

In [5]:
# Crawl Data from internet to get result on a league for a specific season
def crawlLeagueBySeason( season, league ):
    url = website+"/"+season+"/"+league+".csv"
    req = requests.get(url).content
    df = pd.read_csv(io.StringIO(req.decode('utf-8')))
    # Remove not use columns only for premier league
    if 'Attendance' in df.columns:
        df = df[df.columns.drop(['Attendance'])]
    if 'Referee' in df.columns:
        df = df[df.columns.drop(['Referee'])]
    if 'SBH' in df.columns:
        df = df[df.columns.drop(['SBH'])]
    if 'SBD' in df.columns:
        df = df[df.columns.drop(['SBD'])]
    if 'SBA' in df.columns:
        df = df[df.columns.drop(['SBA'])]
    if 'GBH' in df.columns:
        df = df[df.columns.drop(['GBH'])]
    if 'GBA' in df.columns:
        df = df[df.columns.drop(['GBA'])]
    if 'GBD' in df.columns:
        df = df[df.columns.drop(['GBD'])]
    if 'BSH' in df.columns:
        df = df[df.columns.drop(['BSH'])]
    if 'BSD' in df.columns:
        df = df[df.columns.drop(['BSD'])]
    if 'BSA' in df.columns:
        df = df[df.columns.drop(['BSA'])]
    if 'Unnamed: 70' in df.columns:
        df = df[df.columns.drop(['Unnamed: 70'])]
    if 'Unnamed: 71' in df.columns:
        df = df[df.columns.drop(['Unnamed: 71'])]
    if 'Unnamed: 72' in df.columns:
        df = df[df.columns.drop(['Unnamed: 72'])]
    # Transform Date column to Date Type
    if len(df['Date'][0]) == 8:
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
    else:
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    # Insert to DB
    df.to_sql("cur_season_matchs_raw", conn, if_exists="append")
    display(league, season,100. * df.FTR.value_counts() / len(df.FTR))

In [6]:
# Crawl all seasons for all leagues
for league in leagues:
    for season in seasons:
        crawlLeagueBySeason(season, league)

'D1'

'1718'

H    46.031746
D    28.571429
A    25.396825
Name: FTR, dtype: float64

'E0'

'1718'

...

'1718'

H    45.714286
A    30.714286
D    23.571429
Name: FTR, dtype: float64

'SC0'

'1718'

A    42.553191
H    31.914894
D    25.531915
Name: FTR, dtype: float64

In [7]:
# Get all data for pre match on away team
def homeData( date, team, div, nb_matches, nb_matches_string ):
    # Dataframe to return with all info
    dic = {}
    # Home team query
    #TODO Recuperer aussi combien de buts ils se sont pris dans la tronche et tout et tout
    queryHome = '''
            SELECT Date, FTHG, FTR, HTHG, HTR, HS, HST, HF, HC, HY, HR, FTAG, HTAG, `AS`, AST, AF, AC, AY, AR
            FROM cur_season_matchs_raw
            WHERE Date < ? AND HomeTeam = ? AND Div = ? ORDER BY Date DESC LIMIT ?'''
    # Get the previous home game of the Home Team
    df_home = pd.read_sql(queryHome, conn, params=[date, team, div, nb_matches])
    # Hot-encode Category Full Time Result and Half Time Result
    df_home = pd.get_dummies(df_home, columns=['FTR', 'HTR'])
    # Calculate the mean of all columns
    #display(df_home.head())
    dic['H_MEANS_'+nb_matches_string+'_FTHG'] = round(df_home.FTHG.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_FTR_H'] = 0 if 'FTR_H'not in df_home.columns else round(df_home.FTR_H.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_FTR_D'] = 0 if 'FTR_D' not in df_home.columns else round(df_home.FTR_D.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_FTR_A'] = 0 if 'FTR_A' not in df_home.columns else round(df_home.FTR_A.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTHG'] = round(df_home.HTHG.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTR_H'] = 0 if 'HTR_H' not in df_home.columns else round(df_home.HTR_H.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTR_D'] = 0 if 'HTR_D' not in df_home.columns else round(df_home.HTR_D.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTR_A'] = 0 if 'HTR_A' not in df_home.columns else round(df_home.HTR_A.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HS'] = round(df_home.HS.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HST'] = round(df_home.HST.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HF'] = round( df_home.HF.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HC'] = round(df_home.HC.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HY'] = round(df_home.HY.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HR'] = round(df_home.HR.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_FTAG'] = round(df_home.FTAG.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_HTAG'] = round(df_home.HTAG.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AS'] = round(df_home.AS.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AST'] = round(df_home.AST.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AF'] = round(df_home.AF.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AC'] = round(df_home.AC.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AY'] = round(df_home.AY.mean(), 2)
    dic['H_MEANS_'+nb_matches_string+'_AR'] = round(df_home.AR.mean(), 2)
    dic['H_STD_'+nb_matches_string+'_FTHG'] = round(df_home.FTHG.std(), 3)
    dic['H_STD_'+nb_matches_string+'_FTR_H'] = 0 if 'FTR_H'not in df_home.columns else round(df_home.FTR_H.std(), 3)
    dic['H_STD_'+nb_matches_string+'_FTR_D'] = 0 if 'FTR_D' not in df_home.columns else round(df_home.FTR_D.std(), 3)
    dic['H_STD_'+nb_matches_string+'_FTR_A'] = 0 if 'FTR_A' not in df_home.columns else round(df_home.FTR_A.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HTHG'] = round(df_home.HTHG.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HTR_H'] = 0 if 'HTR_H' not in df_home.columns else round(df_home.HTR_H.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HTR_D'] = 0 if 'HTR_D' not in df_home.columns else round(df_home.HTR_D.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HTR_A'] = 0 if 'HTR_A' not in df_home.columns else round(df_home.HTR_A.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HS'] = round(df_home.HS.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HST'] = round(df_home.HST.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HF'] = round( df_home.HF.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HC'] = round(df_home.HC.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HY'] = round(df_home.HY.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HR'] = round(df_home.HR.std(), 3)
    dic['H_STD_'+nb_matches_string+'_FTAG'] = round(df_home.FTAG.std(), 3)
    dic['H_STD_'+nb_matches_string+'_HTAG'] = round(df_home.HTAG.std(), 3)
    dic['H_STD_'+nb_matches_string+'_AS'] = round(df_home.AS.std(), 3)
    dic['H_STD_'+nb_matches_string+'_AST'] = round(df_home.AST.std(), 3)
    dic['H_STD_'+nb_matches_string+'_AF'] = round(df_home.AF.std(), 3)
    dic['H_STD_'+nb_matches_string+'_AC'] = round(df_home.AC.std(), 3)
    dic['H_STD_'+nb_matches_string+'_AY'] = round(df_home.AY.std(), 3)
    dic['H_STD_'+nb_matches_string+'_AR'] = round(df_home.AR.std(), 3)
    return dic

In [8]:
# Get all data for pre match on away team
def awayData( date, team, div, nb_matches, nb_matches_string ):
    # Dataframe to return with all info
    dic = {}
    # away team query
    #TODO Recuperer aussi combien de buts ils se sont pris dans la tronche et tout et tout
    queryAway = '''
            SELECT Date, FTAG, FTR, HTAG, HTR, `AS`, AST, AF, AC, AY, AR, FTHG, HTHG, HS, HST, HF, HC, HY, HR
            FROM cur_season_matchs_raw
            WHERE Date < ? AND AwayTeam = ? AND Div = ? ORDER BY Date DESC LIMIT ?'''
    # Get the previous away game of the away Team
    df_away = pd.read_sql(queryAway, conn, params=[date, team, div, nb_matches])
    # Hot-encode Category Full Time Result and Half Time Result
    df_away = pd.get_dummies(df_away, columns=['FTR', 'HTR'])
    # Calculate the mean of all columns
    #display(df_away.head())
    dic['A_MEANS_'+nb_matches_string+'_FTAG'] = round(df_away.FTAG.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_FTR_H'] = 0 if 'FTR_H'not in df_away.columns else round(df_away.FTR_H.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_FTR_D'] = 0 if 'FTR_D' not in df_away.columns else round(df_away.FTR_D.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_FTR_A'] = 0 if 'FTR_A' not in df_away.columns else round(df_away.FTR_A.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTAG'] = round(df_away.HTAG.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTR_H'] = 0 if 'HTR_H' not in df_away.columns else round(df_away.HTR_H.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTR_D'] = 0 if 'HTR_D' not in df_away.columns else round(df_away.HTR_D.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTR_A'] = 0 if 'HTR_A' not in df_away.columns else round(df_away.HTR_A.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AS'] = round(df_away.AS.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AST'] = round(df_away.AST.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AF'] = round(df_away.AF.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AC'] = round(df_away.AC.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AY'] = round(df_away.AY.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_AR'] = round(df_away.AR.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_FTHG'] = round(df_away.FTHG.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HTHG'] = round(df_away.HTHG.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HS'] = round(df_away.HS.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HST'] = round(df_away.HST.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HF'] = round( df_away.HF.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HC'] = round(df_away.HC.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HY'] = round(df_away.HY.mean(), 2)
    dic['A_MEANS_'+nb_matches_string+'_HR'] = round(df_away.HR.mean(), 2)
    dic['A_STD_'+nb_matches_string+'_FTAG'] = round(df_away.FTAG.std(), 3)
    dic['A_STD_'+nb_matches_string+'_FTR_H'] = 0 if 'FTR_H'not in df_away.columns else round(df_away.FTR_H.std(), 3)
    dic['A_STD_'+nb_matches_string+'_FTR_D'] = 0 if 'FTR_D' not in df_away.columns else round(df_away.FTR_D.std(), 3)
    dic['A_STD_'+nb_matches_string+'_FTR_A'] = 0 if 'FTR_A' not in df_away.columns else round(df_away.FTR_A.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HTAG'] = round(df_away.HTAG.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HTR_H'] = 0 if 'HTR_H' not in df_away.columns else round(df_away.HTR_H.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HTR_D'] = 0 if 'HTR_D' not in df_away.columns else round(df_away.HTR_D.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HTR_A'] = 0 if 'HTR_A' not in df_away.columns else round(df_away.HTR_A.std(), 3)
    dic['A_STD_'+nb_matches_string+'_AS'] = round(df_away.AS.std(), 3)
    dic['A_STD_'+nb_matches_string+'_AST'] = round(df_away.AST.std(), 3)
    dic['A_STD_'+nb_matches_string+'_AF'] = round(df_away.AF.std(), 3)
    dic['A_STD_'+nb_matches_string+'_AC'] = round(df_away.AC.std(), 3)
    dic['A_STD_'+nb_matches_string+'_AY'] = round(df_away.AY.std(), 3)
    dic['A_STD_'+nb_matches_string+'_AR'] = round(df_away.AR.std(), 3)
    dic['A_STD_'+nb_matches_string+'_FTHG'] = round(df_away.FTHG.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HTHG'] = round(df_away.HTHG.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HS'] = round(df_away.HS.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HST'] = round(df_away.HST.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HF'] = round( df_away.HF.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HC'] = round(df_away.HC.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HY'] = round(df_away.HY.std(), 3)
    dic['A_STD_'+nb_matches_string+'_HR'] = round(df_away.HR.std(), 3)
    return dic

In [9]:
# Compute all pre matchs and insert them in pre_match table
df_macths_raw = pd.read_sql_query("SELECT Div, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HTHG, HTAG, HTR, HS, `AS`, HST, AST, HF, AF, HC, AC, HY, AY, HR, AR, PSH, PSD, PSA, BbAvH, BbAvD, BbAvA FROM cur_season_matchs_raw ORDER BY Date ASC;", conn)
for index, row in df_macths_raw.iterrows():
    #display(row)
    
    # HOME TEAM
    # Get the 5 previous home game of the Home Team
    dic_home_five = homeData(row.Date, row.HomeTeam, row.Div, 5, 'FIVE')
    #display(df_home_five)
    # Get the 3 previous home game of the Home Team
    dic_home_three = homeData(row.Date, row.HomeTeam, row.Div, 3, 'THREE')
    #display(df_home_three)
    
    # AWAY TEAM
    # Get the 5 previous away game of the Away Team
    dic_away_five = awayData(row.Date, row.AwayTeam, row.Div, 5, 'FIVE')
    #display(df_away_five)
    # Get the 3 previous away game of the Away Team
    dic_away_three = awayData(row.Date, row.AwayTeam, row.Div, 3, 'THREE')
    #display(df_away_three)
    
    # Merge different dataset together
    #dic_all = dic_home_five.copy().update(dic_home_three)
    dic_all = dict(dic_home_five.items() + dic_home_three.items() + dic_away_five.items() + dic_away_three.items())
    # Add division and date
    dic_all['INFO_Div'] = row['Div']
    dic_all['INFO_Date'] = row['Date']
    # Add info on result
    dic_all['INFO_HomeTeam'] = row['HomeTeam']
    dic_all['INFO_AwayTeam'] = row['AwayTeam']
    dic_all['INFO_FTR'] = row['FTR']
    dic_all['INFO_HTR'] = row['HTR']
    dic_all['INFO_FTHG'] = row['FTHG']
    dic_all['INFO_FTAG'] = row['FTAG']
    # Add all Bet
    dic_all['INFO_PSH'] = row['PSH']
    dic_all['INFO_PSD'] = row['PSD']
    dic_all['INFO_PSA'] = row['PSA']
    dic_all['INFO_BbAvH'] = row['BbAvH']
    dic_all['INFO_BbAvD'] = row['BbAvD']
    dic_all['INFO_BbAvA'] = row['BbAvA']
    df_pre_matches = pd.DataFrame.from_dict([dic_all], orient='columns')
    df_pre_matches.to_sql("cur_season_pre_matchs", conn, if_exists="append")

df_macths_raw.shape

(1295, 28)